# Praca domowa

Praca domowa polega na wykonaniu za pomocą notebooków "pseudoaplikacji", która będzie sugerowała aktywności dla znudzonych $n$ osób. W pracy domowej proszę wykorzystać Bored API (http://www.boredapi.com/). Poniżej znajduje się przykładowe wywołanie tego API. Potrzebne jest zainstalowanie biblioteki `requests` (ale ją powinno Państwo mieć). W przykładzie wysyłamy rządanie do serwera a on zwraca nam odpowiedź. Odpowiedz konwertujemy do postaci Pythonowskiego słownika i możemy z nią dalej pracować:

In [0]:
import requests
# Możliwe typy aktywności 
# ["education", "recreational", "social", "diy", "charity", "cooking", "relaxation", "music", "busywork"]

url = 'https://www.boredapi.com/api/activity?type=recreational&participants=2'
activity = requests.get(url).json()
activity

{'accessibility': 0.4,
 'activity': 'Play a game of tennis with a friend',
 'key': '1093640',
 'link': '',
 'participants': 2,
 'price': 0.1,
 'type': 'recreational'}

Jak widzimy API zwraca nam słownik, w którym mamy informacje o aktywności. Drugim elementem zadania jest wykorzystanie widgetów. 

Aby zainstalować i uruchomić widgety należy posłużyć się poleceniem:

```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

lub 

```
conda install -c conda-forge ipywidgets
```

Przykładowo możemy skonstruować widgety, którymi kontrolować będziemy liczbę uczestników aktywności i typ. Poniżej znajduje się prosty przykład wykorzystania widgetów w Notebooku. Najpierw tworzymy 3 widgety - listę z wyborem aktywności, suwak z wyborem liczby uczestników oraz przycisk akceptacji wyboru. Przycisk akceptacji wyboru podłączony jest pod funkcję `on_click_callback`, która wyświetla aktualnie wybrane wartości pozostałych widgetów (atrybut `value`)

In [0]:
import ipywidgets as widgets
from IPython.display import display
n_participants = widgets.IntSlider(min = 0, max = 10)
types_of_activities = widgets.RadioButtons(options = ["education", "recreational", "social", 
                                                      "diy", "charity", "cooking", 
                                                      "relaxation", "music", "busywork"])

select_button = widgets.Button(description='Kliknij na mnie!')

def on_click_callback(w):
    display(n_participants.value)
    display(types_of_activities.value)
    
select_button.on_click(on_click_callback)

display('Wybrane opcje')
display(types_of_activities)
display(n_participants)
display(select_button)

'Wybrane opcje'

RadioButtons(options=('education', 'recreational', 'social', 'diy', 'charity', 'cooking', 'relaxation', 'music…

IntSlider(value=0, max=10)

Button(description='Kliknij na mnie!', style=ButtonStyle())

0

'education'

0

'charity'

0

'music'

Państwa zadaniem jest rozwinięcie powyższego przykładu tak, aby ściągał z API niezbędne dane i wyświetlał je w formie sformatowanego Markdowna. Rozwiązania należy wrzucić na platformę w postaciu pliku `ipynb`. Jeżeli komuś nie działają widgety, to proszę o kontakt. W ostateczności może być bez widgetów (ale koniecznie z pobieraniem danych przez API i formatowaniem ich w Markdownie).

In [0]:
import requests
import ipywidgets as widgets
from IPython.display import DisplayHandle, Markdown
n_participants = widgets.IntSlider(min = 0, max = 10)
types_of_activities = widgets.RadioButtons(options = ["education", "recreational", "social", 
                                                      "diy", "charity", "cooking", 
                                                      "relaxation", "music", "busywork"])
text_box = widgets.Text('Select the type of activity', disabled=True)

select_button = widgets.Button(description='Show me what you got!', button_style='danger', layout=widgets.Layout(width='200px', height='50px'))
url = 'https://www.boredapi.com/api/activity'
handle = DisplayHandle()

def on_click_callback(w):
    num_of_part = n_participants.value
    act_type = types_of_activities.value
    activity = requests.get(url, params={'type': act_type, 'participants': num_of_part}).json()
    act_string = ''
    if 'error' in activity:
      act_string = 'We are sorry! We dont know *{}* activity for *{}* participant(s)'.format(act_type, num_of_part)
      mod = [int((i+2)/2)*(-1)**(i) for i in range(20)] # Tworzę listę [1, -1, 2, -2, ...], żeby sprawdzić, czy dla innej liczby osób się coś nie znajdzie
      for i in mod:
        activity = requests.get(url, params={'type': act_type, 'participants': num_of_part+i}).json()
        if 'error' not in activity:
          break
      m_l = 'more' if i > 0 else 'less'
      act_string = '\n\n'.join([act_string, 'But you can ***{}***, if you have **{} {}** person(s)'.format(activity['activity'], abs(i), m_l)])
    else:
       act_string = 'You can ***{}***'.format(activity['activity'])

    if activity['accessibility'] >= 0.7:
      act_string = '\n\n'.join([act_string, 'Warning! Accessibility threshold is over 9000'])
    if activity['link']:
      act_string = '\n\n'.join([act_string, 'For more information check out this website [link]({})'.format(activity['link'])])
    handle.update(Markdown(act_string))

handle.display(text_box)
handle.display(types_of_activities)
handle.display(n_participants)
handle.display(select_button)
handle.display(Markdown(''))
select_button.on_click(on_click_callback)

Text(value='Select the type of activity', disabled=True)

RadioButtons(options=('education', 'recreational', 'social', 'diy', 'charity', 'cooking', 'relaxation', 'music…

IntSlider(value=0, max=10)

Button(button_style='danger', description='Show me what you got!', layout=Layout(height='50px', width='200px')…